In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
import torch
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset, random_split

In [4]:
transform = transforms.Compose([
	transforms.ToTensor(),
	transforms.Normalize((0.5,), (0.5,))
])

dataset = datasets.ImageFolder('data', transform=transform)

In [5]:
TRAIN_PORTION = 0.8

train_size = int(TRAIN_PORTION * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [6]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [17]:
class FaceClasifier(nn.Module):
	def __init__(self):
		super(FaceClasifier, self).__init__()
		self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1)
		self.conv2 = nn.Conv2d(32, 64, kernel_size=5, stride=2)
		self.pool = nn.MaxPool2d(3, 3)
		self.fc1 = nn.Linear(64 * 32 * 32, 128)
		self.fc2 = nn.Linear(128, 2)
		self.dropout = nn.Dropout(0.25)
		self.relu = nn.LeakyReLU()

	def forward(self, x):
		x = self.pool(self.relu(self.conv1(x)))
		x = self.pool(self.relu(self.conv2(x)))
		x = x.view(-1, 64 * 32 * 32)
		x = self.relu(self.fc1(x))
		x = self.fc2(x)
		return x

In [18]:
model = FaceClasifier()

In [19]:
for name, param in model.named_parameters():
	# print parameter name and size
	print(name, param.numel())
print('------------------')
print(f'Total parameters: {sum(p.numel() for p in model.parameters()):,}')

conv1.weight 864
conv1.bias 32
conv2.weight 51200
conv2.bias 64
fc1.weight 8388608
fc1.bias 128
fc2.weight 256
fc2.bias 2
------------------
Total parameters: 8441154


In [20]:
criteria = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [21]:
num_epochs = 10
from collections import defaultdict

train_losses = defaultdict(list)

for epoch in range(num_epochs):
	running_loss = 0.0
	for i, (inputs, label) in enumerate(train_loader, 1):
		optimizer.zero_grad()
		outputs = model(inputs)
		loss = criteria(outputs, label)
		loss.backward()
		optimizer.step()
		running_loss += loss.item()
		if i % 10 == 0:
			print(f'Epoch: {epoch + 1}, Batch: {i}, Loss: {running_loss / i}')
			train_losses['batch'].append(i + epoch * len(train_loader))
			train_losses['loss'].append(running_loss / i)
			running_loss = 0.0

print('Finished Training')

Epoch: 1, Batch: 10, Loss: 1.2046668350696563
Epoch: 1, Batch: 20, Loss: 0.3468789756298065
Epoch: 2, Batch: 10, Loss: 0.6788401305675507
Epoch: 2, Batch: 20, Loss: 0.33881733417510984
Epoch: 3, Batch: 10, Loss: 0.6719120919704438
Epoch: 3, Batch: 20, Loss: 0.323357480764389
Epoch: 4, Batch: 10, Loss: 0.6132462382316589
Epoch: 4, Batch: 20, Loss: 0.32422320246696473
Epoch: 5, Batch: 10, Loss: 0.6445241749286652
Epoch: 5, Batch: 20, Loss: 0.304659315943718
Epoch: 6, Batch: 10, Loss: 0.5809789955615997
Epoch: 6, Batch: 20, Loss: 0.28673924654722216
Epoch: 7, Batch: 10, Loss: 0.572023457288742
Epoch: 7, Batch: 20, Loss: 0.2814085423946381
Epoch: 8, Batch: 10, Loss: 0.5039528787136078


In [ ]:
# test accuracy
correct = 0
total = 0
for inputs, label in test_loader:
	outputs = model(inputs)
	_, predicted = torch.max(outputs.data, 1)
	total += label.size(0)
	correct += (predicted == label).sum().item()

print(f'Accuracy of the network on the {total} test images: {100 * correct / total:.2f}%')